In [18]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
import dotenv 

dotenv.load_dotenv()

True

## Step 1. Load
Specify a DocumentLoader to load in your unstructured data as Documents.

A Document is a `dict` with text (page_content) and metadata.

In [27]:
from src.datasets.downloader import DatasetDownloader

# Initialize
downloader = DatasetDownloader()

# Download asset_price
file_path = downloader.download('narcotrafico_chile_2022')

# Get local path
data_path = downloader.get_local_path('narcotrafico_chile_2022')
print(data_path)


../data/raw/narcotrafico_chile_2022


In [28]:
# Load documents
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(data_path)
data = loader.load()

## Step 2. Split
Split the Document into chunks for embedding and vector storage.

In [30]:
# Split documents
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
splits = text_splitter.split_documents(loader.load())

## Step 3. Store
To be able to look up our document splits, we first need to store them where we can later look them up.

The most common way to do this is to embed the contents of each document split.

We store the embedding and splits in a vectorstore.



In [7]:
# Embed and store splits
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=splits,embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

## Step 4. Retrieve
Retrieve relevant splits for any question using similarity search.

This is simply "top K" retrieval where we select documents based on embedding similarity to the query.

In [8]:
# Prompt 
# https://smith.langchain.com/hub/rlm/rag-prompt
from langchain import hub

rag_prompt = hub.pull("rlm/rag-prompt")
rag_prompt

ChatPromptTemplate(input_variables=['question', 'context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question', 'context'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [9]:
# LLM
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [10]:
# RAG chain 

from langchain.schema.runnable import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | rag_prompt 
    | llm 
)

In [11]:
rag_chain.invoke("Ha aumentado el narcotrafico en Chile?")

AIMessage(content='Sí, ha aumentado el narcotráfico en Chile. Se han registrado incautaciones de droga en el puerto de San Antonio en los últimos dos años. Además, se ha observado un incremento en el tráfico de drogas de síntesis y en la cantidad de cocaína incautada por Carabineros de Chile en el año 2021.')

In [12]:
rag_chain.invoke("Cual es el factor principal que explica el aumento en el narcotrafico en Chile?")

AIMessage(content='El factor principal que explica el aumento en el narcotráfico en Chile es la extensa frontera terrestre y marítima que es difícil de monitorear exhaustivamente, lo que aumenta las probabilidades de éxito para quienes intentan internar drogas. Además, la cercanía de Chile con dos de los principales productores de estupefacientes en la región también contribuye a la expansión del narcotráfico en el país.')